In [23]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import functions_forcrawl

edit = functions_forcrawl.All_func()

In [24]:
def clean_semicolon(st):
    
    if re.search(';$|; $', st, flags=re.S):
        return re.sub(';$|; $', '', st, flags=re.S).strip()
    
    else:
        return st
    
def add_minutes(t, m):
    
    hours, minutes = map(int, t.split(":"))
    total_minutes = 60 * hours + minutes + m
    new_hours = total_minutes // 60
    new_minutes = total_minutes % 60
    return "{:02d}:{:02d}".format(new_hours % 24, new_minutes)


In [11]:
import requests

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://eccmid2023.key4.live',
    'Referer': 'https://eccmid2023.key4.live/programme-live-1?coday=2023-04-15&embed=1',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': '3420',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_blank',
    'embed': 'true',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post('https://eccmid2023.key4.live/fo-user-display-session-details.php', headers=headers, data=data)

In [25]:
import requests

cookies = {
    'PHPSESSID': 'lbwww6~2f22d77a3de3da12104acedee740dc52',
    'joe-chnlcustid': '0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
}

headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'PHPSESSID=lbwww6~2f22d77a3de3da12104acedee740dc52; joe-chnlcustid=0a43eed3-8ddd-4ab1-a6c7-c5256b4d9c49',
    'Origin': 'https://online.eccmid.org',
    'Referer': 'https://online.eccmid.org/programme-live-1?coday=2023-02-16&embed=',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'idCat': '1',
    'sessionRef': '3379',
    'timezone': 'Europe/Paris',
    'defaultTimezone': 'Europe/Paris',
    'dispCountry': '',
    'target': '_self',
    'embed': 'false',
    'dispCities': '',
    'firstnameFull': '',
    'page': '',
}

response = requests.post(
    'https://online.eccmid.org/fo-user-display-session-details.php',
    cookies=cookies,
    headers=headers,
    data=data,
)

In [4]:
dates = ['02-15', '02-16', '03-15', '03-16', '04-15', '04-16', '04-17', '04-18', '05-24', '05-25']

In [28]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_number':[],
    'name':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [29]:
se_no = 1
for dat in dates:
    
    reqout = requests.get(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
    soup = BS(reqout.content, 'lxml')
    
    outer_row = soup.find_all(class_='row live-program-card-row-grid')
    locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')
    
    for ou in outer_row:

        col_div = 0
        
        for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
            
            if i_id.find(class_='programme-session-card programme-session-display border'):
                
                session_number = f'S{se_no}'
                print(f'=====**====={session_number}=====**=======', col_div)
                i_id = i_id.find(class_='programme-session-card programme-session-display border')

                data['sessionRef'] = i_id.get('data-session-reference')
                
                req = requests.post(
                    'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                    cookies=cookies,
                    headers=headers,
                    data=data,)

                soupin = BS(req.content, 'lxml')
                sess_data = soupin.find(id='modal-session-content')

                # -------------------------------------    date time location ya type     ------------------------------------
                sess_date = ''
                sess_time = ''
                sess_loc = ''
                sess_type = ''
                sess_loc = ''
                sess_loc = locations[col_div].text.strip()
                date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                sess_date = date_time_type.split(',')[0].strip()
                sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                sess_type = date_time_type.split('CET')[-1].strip()

                # --------------------------------------  category or type and title -----------------------------------------------------
                sess_cate = ''
                sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                # --------------------------------------- abstract text OR co organised ---------------------------------------------
                co_org = ''
                sess_text = ''
                if sess_data.find(class_='modal-session-organized-by mt-2'):
                    co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                if sess_data.find(class_='modal-session-desc mt-2'):
                    sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()
                    
                sess_text = co_org + ' ' + sess_text

                # ------------------------------------------- session ka authors aff --------------------------------------------------
                sess_author = ''
                sess_aff = ''

                if sess_data.find(class_='modal-session-moderators'):
                    for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                        aff = i.find(class_='modal-session-moderator-country').text.strip()
                        auth = i.text.strip()
                        auth = auth.replace(aff, '').split('-')[0]
                        aff = aff.replace(',', '').strip()

                        sess_author += auth + '; '
                        sess_aff += aff + '; '

                sess_author = clean_semicolon(sess_author)
                sess_aff = clean_semicolon(sess_aff)
                sess_date = sess_date + '/2023'

                print(f'sess date :- {sess_date}')
                print(f'sess time :- {sess_time}')
                print(f'sess loca :- {sess_loc}')
                print(f'sess type :- {sess_type}')
                print(f'sess cate :- {sess_cate}')
                print(f'sess title :- {sess_title}')
                print(f'abs text :- {sess_text}')
                print(f'sess auth :- {sess_author}')
                print(f'sess aff :- {sess_aff}')

                dic['session_number'].append(session_number)
                dic['name'].append('Session')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(sess_title)
                dic['url'].append(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
                dic['authors'].append(sess_author)
                dic['author_affiliation'].append(sess_aff)
                dic['abstract_text'].append(sess_text)
                dic['date'].append(sess_date)
                dic['start_time'].append(sess_time.split('-')[0].strip())
                dic['end_time'].append(sess_time.split('-')[-1].strip())
                dic['location'].append(sess_loc)
                dic['session_title'].append(sess_title)
                dic['session_type'].append(sess_type)
                dic['category'].append(sess_cate)
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                print('======'*20)
                
                # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*
                
                abs_data = sess_data.find(class_='modal-sessions-interventions')
                if abs_data:
                    
                    pno = 1
                    for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                        abs_title = ab.find(style='font-weight: bold').text.strip()

                        # abs author
                        abs_author = ''
                        abs_aff = ''
                        if ab.find(class_='modal-session-faculties mb-1'):
                            ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                            abs_author = ', '.join(ab_authaff.split(',')[:-1])
                            abs_aff = ab_authaff.split(',')[-1]

                        if abs_title == '':
                            abs_title = 'TBD'
                            
                        # source id...... time shit
                        
                        ar_id = ''
                        art_time = ''
                        
                        if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                            src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                            if src_code:

                                if re.search('^\d{1,6}', src_code):
                                    ar_id = src_code + '-' + str(pno)
                                    art_time = ''
                                elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                    str_pno = str(pno)
                                    if len(str_pno) == 1:
                                        str_pno = '0' + str_pno
                                    ar_id = src_code + str_pno
                                    art_time = sess_time
                        else:
                            ar_id = 'QC'
                        
                            
                        print(ar_id, '===', art_time)
                        print(f'abs title :- {abs_title}')
                        print(f'abs auth :- {abs_author}')
                        print(f'abs aff :- {abs_aff}')

                        dic['session_number'].append(session_number)
                        dic['name'].append('Abstract')
                        dic['source_id'].append(ar_id)
                        dic['manual_id'].append('')
                        dic['article_title'].append(abs_title)
                        dic['url'].append(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
                        dic['authors'].append(abs_author)
                        dic['author_affiliation'].append(abs_aff)
                        dic['abstract_text'].append('')
                        dic['date'].append(sess_date)
                        dic['start_time'].append('')
                        dic['end_time'].append('')
                        dic['location'].append(sess_loc)
                        dic['session_title'].append(sess_title)
                        dic['session_type'].append(sess_type)
                        dic['category'].append(sess_cate)
                        dic['sub_category'].append('')
                        dic['disclosure'].append('')
                        print('-*-+-'* 17)
                        pno += 1
                        
                se_no += 1
                print('inr', se_no)
                
            col_div += 1
            

02-15
02-16
03-15
03-16
04-15
=====**=====S1=====**======= 0
sess date :- 15/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 3. Bacterial susceptibility & resistance
sess title :- PS018 - 3a. Resistance surveillance & epidemiology: MRSA, VRE & other Gram-positives
abs text :-  
sess auth :- 
sess aff :- 
PS01801 === 12:00 - 13:30
abs title :- Study of fusidic acid resistance among Staphylococcus aureus skin and soft tissue isolates, eastern India
abs auth :- Srujana MOHANTY
abs aff :-  India
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS01802 === 12:00 - 13:30
abs title :- Ecological changes in Gram-positive cocci bloodstream infections in critical areas after ceftaroline introduction: not everything is so predictable
abs auth :- Ana Patricia GONZALEZ-SAIZ
abs aff :-  Spain
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS01803 === 12:00 - 13:

=====**=====S2=====**======= 0
sess date :- 16/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 2. Bacterial infection & disease
sess title :- PS010 - 2a. Tuberculosis and other mycobacterial infections (incl. epidemiology, clinical, diagnostics, antimycobacterial drugs, susceptibility testing, treatment & prevention)
abs text :-  
sess auth :- 
sess aff :- 
PS01001 === 12:00 - 13:30
abs title :- Did discordance in genotypic and phenotypic sensitivities impact clinical care in tuberculosis patients at a UK centre?
abs auth :- Anees FATIMA
abs aff :-  United Kingdom
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS01002 === 12:00 - 13:30
abs title :- Isoniazid mono-resistant tuberculosis: trends in a tertiary hospital in KwaZulu-Natal province, South Africa
abs auth :- Afsana KAJEE
abs aff :-  South Africa
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--

=====**=====S3=====**======= 0
sess date :- 17/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 10. Immunology & vaccinology
sess title :- PS069 - 10a. Host genetics: infection susceptibility & primary immunodeficiency
abs text :-  
sess auth :- 
sess aff :- 
PS06901 === 12:00 - 13:30
abs title :- Polymorphisms in genes regulating innate immunity and risk for RVVC in Nigerian women
abs auth :- Samuel Adetona FAYEMIWO
abs aff :-  United Kingdom
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
inr 4
04-18
=====**=====S4=====**======= 0
sess date :- 18/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 11. Other
sess title :- PS076 - 11a. Professional affairs and career development
abs text :-  
sess auth :- 
sess aff :- 
PS07601 === 12:00 - 13:30
abs title :- Disproportionate deployment of infectious disease specialists at design

In [30]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_number,name,session_type,category,sub_category,disclosure


### add 30 mins time with pd

In [78]:
sestit = set()
c = 0
for i in df['source_id']:
    i = str(i)
    if re.search('^\d{1,10}', i):
        sestit.add(df.iloc[c]['session_title'])
    c += 1

In [79]:


for tit in sestit:
    c = 0
    grp = []
    for i in df['session_title']:
        if i == tit:
            grp.append(c)
            print(c)
        c += 1
        
    if len(grp) >= 3:

        stime = df.iloc[grp[0]]['start_time']
        entime = df.iloc[grp[0]]['end_time']

        nstim = stime
        netim = add_minutes(stime, 30)

        df.iat[grp[1], df.columns.get_loc('start_time')] = nstim
        df.iat[grp[1], df.columns.get_loc('end_time')] = netim

        for tim in grp[2:-1]:
            print(nstim, netim)

            nstim = add_minutes(nstim, 30)
            netim = add_minutes(netim, 30)

            df.iat[tim, df.columns.get_loc('start_time')] = nstim
            df.iat[tim, df.columns.get_loc('end_time')] = netim

        lst_st = df.iloc[grp[-2]]['end_time']
        lst_en = ''

        if str(add_minutes(df.iloc[grp[-2]]['end_time'], 30)).strip() == str(entime).strip():
            print('same')
            lst_en = entime

        df.iat[grp[-1], df.columns.get_loc('start_time')] = lst_st
        df.iat[grp[-1], df.columns.get_loc('end_time')] = lst_en


    else:
        
        for nc in grp[1:]:
            df.iat[nc, df.columns.get_loc('start_time')] = 'check'
    

572
573
574
575
08:30 09:00
408
409
410
same
1320
1321
1352
1353
1354
same
1741
1742
1743
1744
1745
13:30 14:00
14:00 14:30
same
327
328
329
same
27
28
29
same
1276
1277
1278
same
463
464
465
466
467
16:15 16:45
16:45 17:15
same
414
415
416
same
833
834
835
same
244
245
246
247
248
12:15 12:45
12:45 13:15
399
400
401
same
478
479
480
481
16:15 16:45
969
970
971
972
973
16:15 16:45
16:45 17:15
same
374
375
1334
1335
1336
same
182
183
184
same
1006
1007
1008
1009
16:15 16:45
1674
1675
1676
1677
1678
12:15 12:45
12:45 13:15
1109
1110
1111
1112
08:30 09:00
1617
1618
1619
same
1717
1718
1719
1720
1721
13:30 14:00
14:00 14:30
same
468
469
470
471
472
16:15 16:45
16:45 17:15
same
1722
1723
1724
1725
1726
13:30 14:00
14:00 14:30
same
1173
1174
1175
same
576
577
578
579
580
08:30 09:00
09:00 09:30
same
964
965
966
967
968
16:15 16:45
16:45 17:15
same
1527
1528
1529
1530
08:30 09:00
111
112
113
114
115
08:30 09:00
09:00 09:30
same
1400
1401
1402
1403
1404
16:15 16:45
16:45 17:15
same
21
22
23
sa

In [80]:
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_number,name,session_type,category,sub_category,disclosure
0,NaN,NaN,3437 - Multi-country outbreaks: from signal to...,https://eccmid2023.key4.live/programme-live-1?...,Jacob MORAN; Ines STEFFENS,Israel; Sweden,Co-organised with \nEurosur...,15/02/2023,16:00,17:00,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Session,1-hour SymposiumScience track,7. Parasitic diseases & global health,NaN,NaN
1,3437-1,NaN,A dangerous pre-Easter surprise: multi-country...,https://eccmid2023.key4.live/programme-live-1?...,Johanna TAKKINEN,Sweden,NaN,15/02/2023,16:00,16:30,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Presentation,1-hour SymposiumScience track,7. Parasitic diseases & global health,NaN,NaN
2,3437-2,NaN,Multi-country detections of diphtheria in migr...,https://eccmid2023.key4.live/programme-live-1?...,Ole WICHMANN,Germany,NaN,15/02/2023,16:30,17:00,Science track,3437 - Multi-country outbreaks: from signal to...,S1,Presentation,1-hour SymposiumScience track,7. Parasitic diseases & global health,NaN,NaN
3,NaN,NaN,"3268 - Tick-borne diseases in Europe: new, eme...",https://eccmid2023.key4.live/programme-live-1?...,Nanna Skaarup ANDERSEN ; Olli VAPALAHTI,Denmark; Finland,Co-organised with \nDanish ...,15/02/2023,17:00,18:00,Science track,"3268 - Tick-borne diseases in Europe: new, eme...",S2,Session,1-hour SymposiumScience track,2. Bacterial infection & disease,NaN,NaN
4,3268-1,NaN,Lyme borreliosis: recent progress and remainin...,https://eccmid2023.key4.live/programme-live-1?...,Randi EIKELAND,Norway,NaN,15/02/2023,17:00,17:30,Science track,"3268 - Tick-borne diseases in Europe: new, eme...",S2,Presentation,1-hour SymposiumScience track,2. Bacterial infection & disease,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,3440-1,NaN,Use of rapid diagnostics in ARTIs: a consensus...,https://eccmid2023.key4.live/programme-live-1?...,Murat AKOVA,Turkey,NaN,25/05/2023,17:00,17:30,Science track,3440 - Rapid diagnostics to inform the managem...,S344,Presentation,1-hour SymposiumScience track,"5. New antibacterial agents, PK/PD & Stewardship",NaN,NaN
1811,3440-2,NaN,Implementation of the consensus report in the ...,https://eccmid2023.key4.live/programme-live-1?...,Eva POLVERINO,Spain,NaN,25/05/2023,17:30,18:00,Science track,3440 - Rapid diagnostics to inform the managem...,S344,Presentation,1-hour SymposiumScience track,"5. New antibacterial agents, PK/PD & Stewardship",NaN,NaN
1812,NaN,NaN,3314 - New developments in clinical radiology ...,https://eccmid2023.key4.live/programme-live-1?...,NaN,NaN,Co-organised with \nESCMID ...,25/05/2023,18:00,19:00,Science track,3314 - New developments in clinical radiology ...,S345,Session,Meet-the-ExpertScience track,2. Bacterial infection & disease,NaN,NaN
1813,3314-1,NaN,TBD,https://eccmid2023.key4.live/programme-live-1?...,Barbara HASSE,Switzerland,NaN,25/05/2023,18:00,18:30,Science track,3314 - New developments in clinical radiology ...,S345,Presentation,Meet-the-ExpertScience track,2. Bacterial infection & disease,NaN,NaN


### scrape with filter one by one

In [26]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_number':[],
    'name':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

dates = ['02-15', '02-16', '03-15', '03-16', '04-15', '04-16', '04-17', '04-18', '05-24', '05-25']


In [27]:
dates = ['02-15', '02-16', '03-15', '03-16']

In [28]:
se_no = 1
for dat in dates:
    print(dat)
    reqout = requests.get(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')

    for fil in range(2, 100):
        
#         headers = {
#             'Accept': 'text/html, */*; q=0.01',
#             'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
#             'Connection': 'keep-alive',
#             'Referer': 'https://eccmid2023.key4.live/programme-live-1?&coday=2023-04-17&embed=1',
#             'Sec-Fetch-Dest': 'empty',
#             'Sec-Fetch-Mode': 'cors',
#             'Sec-Fetch-Site': 'same-origin',
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
#             'X-Requested-With': 'XMLHttpRequest',
#             'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
#             'sec-ch-ua-mobile': '?0',
#             'sec-ch-ua-platform': '"Windows"',
#         }

        params = {
            'coday': f'2023-{dat}',
            'embed': '1',
            'advSearchThemeList': f'{fil}' }

        reqout = requests.get('https://eccmid2023.key4.live/programme-live-1', params=params, headers=headers)
        soup = BS(reqout.content, 'lxml')

        outer_row = soup.find_all(class_='row live-program-card-row-grid')
        locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')

        for ou in outer_row:

            col_div = 0

            for i_id in ou.find_all(class_='border text-center program-grid-calandar'):

                if i_id.find(class_='programme-session-card programme-session-display border'):

                    session_number = f'S{se_no}'
                    print(f'=====**====={session_number}=====**=======', col_div)
                    i_id = i_id.find(class_='programme-session-card programme-session-display border')

                    data['sessionRef'] = i_id.get('data-session-reference')

                    req = requests.post(
                        'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                        cookies=cookies,
                        headers=headers,
                        data=data,)

                    soupin = BS(req.content, 'lxml')
                    sess_data = soupin.find(id='modal-session-content')

                    # -------------------------------------    date time location ya type     ------------------------------------
                    sess_date = ''
                    sess_time = ''
                    sess_loc = ''
                    sess_type = ''
                    sess_loc = ''
                    sess_loc = locations[col_div].text.strip()
                    date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                    sess_date = date_time_type.split(',')[0].strip()
                    sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                    sess_type = date_time_type.split('CET')[-1].strip()

                    # --------------------------------------  category or type and title -----------------------------------------------------
                    sess_cate = ''
                    sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                    sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                    sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                    # --------------------------------------- abstract text OR co organised ---------------------------------------------
                    co_org = ''
                    sess_text = ''
                    if sess_data.find(class_='modal-session-organized-by mt-2'):
                        co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                    if sess_data.find(class_='modal-session-desc mt-2'):
                        sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()

                    sess_text = co_org + ' ' + sess_text

                    # ------------------------------------------- session ka authors aff --------------------------------------------------
                    sess_author = ''
                    sess_aff = ''

                    if sess_data.find(class_='modal-session-moderators'):
                        for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                            aff = i.find(class_='modal-session-moderator-country').text.strip()
                            auth = i.text.strip()
                            auth = auth.replace(aff, '').split('-')[0]
                            aff = aff.replace(',', '').strip()

                            sess_author += auth + '; '
                            sess_aff += aff + '; '

                    sess_author = clean_semicolon(sess_author)
                    sess_aff = clean_semicolon(sess_aff)
                    sess_date = sess_date + '/2023'
                    sess_type = re.sub(sess_loc, '', sess_type)
                    
                    print(f'sess date :- {sess_date}')
                    print(f'sess time :- {sess_time}')
                    print(f'sess loca :- {sess_loc}')
                    print(f'sess type :- {sess_type}')
                    print(f'sess cate :- {sess_cate}')
                    print(f'sess title :- {sess_title}')
                    print(f'abs text :- {sess_text}')
                    print(f'sess auth :- {sess_author}')
                    print(f'sess aff :- {sess_aff}')

                    dic['session_number'].append(session_number)
                    dic['name'].append('Session')
                    dic['source_id'].append('')
                    dic['manual_id'].append('')
                    dic['article_title'].append(sess_title)
                    dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                    dic['authors'].append(sess_author)
                    dic['author_affiliation'].append(sess_aff)
                    dic['abstract_text'].append(sess_text)
                    dic['date'].append(sess_date)
                    dic['start_time'].append(sess_time.split('-')[0].strip())
                    dic['end_time'].append(sess_time.split('-')[-1].strip())
                    dic['location'].append(sess_loc)
                    dic['session_title'].append(sess_title)
                    dic['session_type'].append(sess_type)
                    dic['category'].append(sess_cate)
                    dic['sub_category'].append('')
                    dic['disclosure'].append('')
                    print('======'*20)

                    # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*

                    abs_data = sess_data.find(class_='modal-sessions-interventions')
                    if abs_data:

                        pno = 1
                        for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                            abs_title = ab.find(style='font-weight: bold').text.strip()

                            # abs author
                            abs_author = ''
                            abs_aff = ''
                            if ab.find(class_='modal-session-faculties mb-1'):
                                ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                                abs_author = ', '.join(ab_authaff.split(',')[:-1])
                                abs_aff = ab_authaff.split(',')[-1]

                            if abs_title == '':
                                abs_title = 'TBD'

                            # source id...... time shit

                            ar_id = ''
                            art_time = ''

                            if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                                src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                                if src_code:

                                    if re.search('^\d{1,6}', src_code):
                                        ar_id = src_code + '-' + str(pno)
                                        art_time = ''
                                    elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                        str_pno = str(pno)
                                        if len(str_pno) == 1:
                                            str_pno = '0' + str_pno
                                        ar_id = src_code + str_pno
                                        art_time = sess_time
                            else:
                                ar_id = 'QC'


                            print(ar_id, '===', art_time)
                            print(f'abs title :- {abs_title}')
                            print(f'abs auth :- {abs_author}')
                            print(f'abs aff :- {abs_aff}')

                            dic['session_number'].append(session_number)
                            dic['name'].append('Abstract')
                            dic['source_id'].append(ar_id)
                            dic['manual_id'].append('')
                            dic['article_title'].append(abs_title)
                            dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                            dic['authors'].append(abs_author)
                            dic['author_affiliation'].append(abs_aff)
                            dic['abstract_text'].append('')
                            dic['date'].append(sess_date)
                            dic['start_time'].append(sess_time.split('-')[0].strip())
                            dic['end_time'].append(sess_time.split('-')[-1].strip())
                            dic['location'].append(sess_loc)
                            dic['session_title'].append(sess_title)
                            dic['session_type'].append(sess_type)
                            dic['category'].append(sess_cate)
                            dic['sub_category'].append('')
                            dic['disclosure'].append('')
                            print('-*-+-'* 17)
                            pno += 1

                    se_no += 1
                    print('inr', se_no)

                col_div += 1
            

02-15
02-16
03-15
03-16


In [32]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_number,name,session_type,category,sub_category,disclosure


In [33]:
df1 = pd.read_excel('eccmid2.xlsx')
df2 = pd.read_excel('eccmid3.xlsx')

In [34]:
res = pd.concat([df, df1, df2])

In [36]:
res.to_excel('Filter_all.xlsx', index=False)

In [ ]:
df.to_excel('eccmid1.xlsx', index=False)

In [78]:
df.to_excel('posterswala_eccmid_time.xlsx', index=False)

In [79]:
req = requests.get('https://eccmid2023.key4.live/programme-live-1?coday=2023-04-15&embed=1')
soup = BS(req.content, 'lxml')

In [94]:
se = set()
for i in list(soup.find_all(class_='p-3 program_advanced-search_section')[2].find(class_='row').children):
    if len(str(i)) > 4:
        print(i.find('input').get('data-id'), i.find('input').get('value'))
        se.add(i.find('input').get('value'))

70 10a. Host genetics: infection susceptibility & primary immunodeficiency
71 10b. Infections in immunocompromised hosts (incl. epidemiology, clinical features and diagnosis, treatment, excl. fungal infections)
72 10c. General vaccinology (incl. in vitro and in vivo studies, policy, social aspects)
73 10d. Antiviral vaccines (excl. influenza, respiratory and bloodborne viruses and COVID-19)
74 10e. Antibacterial vaccines
75 10f. Other preventive modalities (incl. new vaccine technologies, antibody-based treatments)
76 10g. Immune response to infection (incl. in vitro, animal and human studies, laboratory assessment of immunity and response, excl. sepsis biomarkers)
89 11a. Professional affairs and career development
90 11b. Publishing, ethics, other academic affairs
91 11c. Medical education for CM/ID
92 11d. Digital health and infection (incl. AI, data mining, informatics)
93 11e. Diversity and equality
94 11f. Advocacy and role of patients
77 12a. Basic science and virology
78 12b. E

In [95]:
for tit in se:
    c = 0
    for i in df['article_title']:
        if re.search(tit, i):
            print('found', c)
        c += 1

found 1641
found 1018
found 3280
found 669
found 912
found 2815
found 3276
found 3237
found 650
found 1047
found 2640
found 2746
found 3226
found 3244
found 2721
found 3573
found 2479
found 3469
found 2622
found 3270
found 3543
found 0
found 1874
found 3315
found 1519
found 1904
found 2730
found 2706
found 702
found 3139
found 1927
found 2856
found 3248
found 2865
found 2403


In [30]:
import PyPDF2
def return_pdf_text(pdf_name):
    pdfFileObj = open(pdf_name, 'rb')

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    abs_text = ''
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        abs_text += pageObj.extractText() + ' '
    
    pdfFileObj.close()
    return abs_text

In [31]:
pdf_text = return_pdf_text('anuj_pdf.pdf')

Xref table not zero-indexed. ID numbers for objects will be corrected.


In [33]:
sara_codes = set()

c = 0
for i in df['session_title']:
    
    if df.iloc[c]['session_title'] == df.iloc[c]['article_title']:
        sara_codes.add(i.split('-')[0].strip())
        
    c += 1

In [34]:
len(sara_codes)

347

In [78]:
len(sara_codes)

220

In [35]:
sara_code_pdf = set()

c = 0
for i in sara_codes:
    if re.search(i, pdf_text, flags=re.S) and re.search(f'{i} ', pdf_text, flags=re.S):
        sara_code_pdf.add(i)
        c += 1
        
print(c)

269


In [11]:
c = 0
co = 0
for i in df['article_title']:
    
    if df.iloc[c]['session_title'] != df.iloc[c]['article_title'] and df.iloc[c]['session_title'].split('-')[0].strip() in sara_code_pdf:
        
        if '(' in i:
            search_text = re.search('(.*)\(.*?\)(.*)', i).group(1)
            search_text = search_text + '.*' + re.search('(.*)\(.*?\)(.*)', i).group(2)
        else:
            search_text = i
            
            
            if len(re.findall(search_text, new_pdf_text, flags=re.S)) == 0:
                co += 1

            else:
                print(len(re.findall(search_text, new_pdf_text, flags=re.S)))


                stin = re.search(search_text, new_pdf_text, flags=re.S).span()[0] - 14
                enin = re.search(search_text, new_pdf_text, flags=re.S).span()[1]
                time_srcid = new_pdf_text[stin:enin]

                time_abs = ''
                srcid = ''
                if re.search('\d{1,2}:\d{1,2}', time_srcid):
                    time_abs = re.search('\d{1,2}:\d{1,2}', time_srcid).group()

                if re.search('\d{3,4}|\d{3,4}-\d{1}', time_srcid):
                    srcid = re.search('\d{3,4}-\d{1}|\d{3,4}', time_srcid).group()

                print(time_abs, '==========', srcid, '--', df.iloc[c]['session_title'].split('-')[0].strip())
                if time_abs == '' or srcid == '':
                    print('checkkkk =============')
    
#                 df.iat[c, df.columns.get_loc('start_time')] = time_abs
#                 df.iat[c, df.columns.get_loc('source_id')] = srcid
    c += 1

NameError: name 'df' is not defined

In [ ]:
df.to_excel('pdf_edited_eccmid.xlsx', index=False)

In [18]:
for ou in outer_row:

    col_div = 0

    for i_id in ou.find_all(class_='border text-center program-grid-calandar'):
#         print('f')
        i_id = i_id.find(class_='programme-session-card programme-session-display border')
        print(i_id.get('data-session-reference'))

PS071


AttributeError: 'NoneType' object has no attribute 'get'

In [12]:
se_no = 1
for dat in dates:
    print(dat)
    reqout = requests.get(f'https://eccmid2023.key4.live/programme-live-1?coday=2023-{dat}&embed=1')
    
    params = {
            'coday': f'2023-04-17',
            'embed': '1',
            'advSearchThemeList': f'{72}' }
    fil = 72

    reqout = requests.get('https://eccmid2023.key4.live/programme-live-1', params=params, headers=headers)
    soup = BS(reqout.content, 'lxml')
    
    outer_row = soup.find_all(class_='row live-program-card-row-grid')
    locations = soup.find_all(class_='d-flex-block text-center roomsName py-2')

    for ou in outer_row:

        col_div = 0

        for i_id in ou.find_all(class_='border text-center program-grid-calandar'):

            if i_id.find(class_='programme-session-card programme-session-display border'):

                session_number = f'S{se_no}'
                print(f'=====**====={session_number}=====**=======', col_div)
                i_id = i_id.find(class_='programme-session-card programme-session-display border')

                data['sessionRef'] = i_id.get('data-session-reference')

                req = requests.post(
                    'https://eccmid2023.key4.live/fo-user-display-session-details.php',
                    cookies=cookies,
                    headers=headers,
                    data=data,)

                soupin = BS(req.content, 'lxml')
                sess_data = soupin.find(id='modal-session-content')

                # -------------------------------------    date time location ya type     ------------------------------------
                sess_date = ''
                sess_time = ''
                sess_loc = ''
                sess_type = ''
                sess_loc = ''
                sess_loc = locations[col_div].text.strip()
                date_time_type = edit.text_cleaner(sess_data.find(class_='modal-session-header').text)

                sess_date = date_time_type.split(',')[0].strip()
                sess_time = date_time_type.split(',')[1].split('CET')[0].strip()
                sess_type = date_time_type.split('CET')[-1].strip()

                # --------------------------------------  category or type and title -----------------------------------------------------
                sess_cate = ''
                sess_cate = sess_data.find(class_='modal-title-cat mt-2 mb-2').text.strip()
                sess_title = sess_data.find(class_='modal-session-name mb-2').text.strip()
                sess_title = i_id.get('data-session-reference') + ' - ' + sess_title

                # --------------------------------------- abstract text OR co organised ---------------------------------------------
                co_org = ''
                sess_text = ''
                if sess_data.find(class_='modal-session-organized-by mt-2'):
                    co_org = sess_data.find(class_='modal-session-organized-by mt-2').text.strip()
                if sess_data.find(class_='modal-session-desc mt-2'):
                    sess_text = sess_data.find(class_='modal-session-desc mt-2').text.strip()

                sess_text = co_org + ' ' + sess_text

                # ------------------------------------------- session ka authors aff --------------------------------------------------
                sess_author = ''
                sess_aff = ''

                if sess_data.find(class_='modal-session-moderators'):
                    for i in sess_data.find(class_='modal-session-moderators').find_all(class_='row'):

                        aff = i.find(class_='modal-session-moderator-country').text.strip()
                        auth = i.text.strip()
                        auth = auth.replace(aff, '').split('-')[0]
                        aff = aff.replace(',', '').strip()

                        sess_author += auth + '; '
                        sess_aff += aff + '; '

                sess_author = clean_semicolon(sess_author)
                sess_aff = clean_semicolon(sess_aff)
                sess_date = sess_date + '/2023'

                print(f'sess date :- {sess_date}')
                print(f'sess time :- {sess_time}')
                print(f'sess loca :- {sess_loc}')
                print(f'sess type :- {sess_type}')
                print(f'sess cate :- {sess_cate}')
                print(f'sess title :- {sess_title}')
                print(f'abs text :- {sess_text}')
                print(f'sess auth :- {sess_author}')
                print(f'sess aff :- {sess_aff}')

                dic['session_number'].append(session_number)
                dic['name'].append('Session')
                dic['source_id'].append('')
                dic['manual_id'].append('')
                dic['article_title'].append(sess_title)
                dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                dic['authors'].append(sess_author)
                dic['author_affiliation'].append(sess_aff)
                dic['abstract_text'].append(sess_text)
                dic['date'].append(sess_date)
                dic['start_time'].append(sess_time.split('-')[0].strip())
                dic['end_time'].append(sess_time.split('-')[-1].strip())
                dic['location'].append(sess_loc)
                dic['session_title'].append(sess_title)
                dic['session_type'].append(sess_type)
                dic['category'].append(sess_cate)
                dic['sub_category'].append('')
                dic['disclosure'].append('')
                print('======'*20)

                # *+*+*+*+*+*+*+*+*+*+*+*+*  ------------------- ABSTRACT DATA ----------------- *+*+*+*+*+*+**+**+*+*+*

                abs_data = sess_data.find(class_='modal-sessions-interventions')
                if abs_data:

                    pno = 1
                    for ab in abs_data.find_all(class_='modal-sessions-interventions-group clearfix'):

                        abs_title = ab.find(style='font-weight: bold').text.strip()

                        # abs author
                        abs_author = ''
                        abs_aff = ''
                        if ab.find(class_='modal-session-faculties mb-1'):
                            ab_authaff = ab.find(class_='modal-session-faculties mb-1').text.strip()
                            abs_author = ', '.join(ab_authaff.split(',')[:-1])
                            abs_aff = ab_authaff.split(',')[-1]

                        if abs_title == '':
                            abs_title = 'TBD'

                        # source id...... time shit

                        ar_id = ''
                        art_time = ''

                        if re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title):
                            src_code = re.search('^[A-Z]{1,5}\d{1,10}|^\d{1,5}', sess_title).group()

                            if src_code:

                                if re.search('^\d{1,6}', src_code):
                                    ar_id = src_code + '-' + str(pno)
                                    art_time = ''
                                elif re.search('^[A-Z]{1,5}\d{1,10}', src_code):

                                    str_pno = str(pno)
                                    if len(str_pno) == 1:
                                        str_pno = '0' + str_pno
                                    ar_id = src_code + str_pno
                                    art_time = sess_time
                        else:
                            ar_id = 'QC'


                        print(ar_id, '===', art_time)
                        print(f'abs title :- {abs_title}')
                        print(f'abs auth :- {abs_author}')
                        print(f'abs aff :- {abs_aff}')

                        dic['session_number'].append(session_number)
                        dic['name'].append('Presentation')
                        dic['source_id'].append(ar_id)
                        dic['manual_id'].append('')
                        dic['article_title'].append(abs_title)
                        dic['url'].append(f'https://online.eccmid.org/programme-live-1?coday=2023-{dat}&embed=&advSearchThemeList={fil}')
                        dic['authors'].append(abs_author)
                        dic['author_affiliation'].append(abs_aff)
                        dic['abstract_text'].append('')
                        dic['date'].append(sess_date)
                        dic['start_time'].append(sess_time.split('-')[0].strip())
                        dic['end_time'].append(sess_time.split('-')[-1].strip())
                        dic['location'].append(sess_loc)
                        dic['session_title'].append(sess_title)
                        dic['session_type'].append(sess_type)
                        dic['category'].append(sess_cate)
                        dic['sub_category'].append('')
                        dic['disclosure'].append('')
                        print('-*-+-'* 17)
                        pno += 1

02-15
=====**=====S1=====**======= 0
sess date :- 17/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 10. Immunology & vaccinology
sess title :- PS071 - 10c. General vaccinology (incl. in vitro and in vivo studies, policy, social aspects)
abs text :-  
sess auth :- 
sess aff :- 
PS07101 === 12:00 - 13:30
abs title :- Stalk antibodies after seasonal influenza vaccination
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07102 === 12:00 - 13:30
abs title :- Vaccine coated microstructures to prevent infectious disease
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07103 === 12:00 - 13:30
abs title :- Establishing the VACCELERATE site network: capacity mapping of clinical trial sites
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07104 

=====**=====S1=====**======= 0
sess date :- 17/04/2023
sess time :- 12:00 - 13:30
sess loca :- Poster Area
sess type :- Poster SessionPoster Area
sess cate :- 10. Immunology & vaccinology
sess title :- PS071 - 10c. General vaccinology (incl. in vitro and in vivo studies, policy, social aspects)
abs text :-  
sess auth :- 
sess aff :- 
PS07101 === 12:00 - 13:30
abs title :- Stalk antibodies after seasonal influenza vaccination
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07102 === 12:00 - 13:30
abs title :- Vaccine coated microstructures to prevent infectious disease
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07103 === 12:00 - 13:30
abs title :- Establishing the VACCELERATE site network: capacity mapping of clinical trial sites
abs auth :- 
abs aff :- 
-*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+--*-+-
PS07104 === 12

KeyboardInterrupt: 